<a href="https://colab.research.google.com/github/FacuML/NLP/blob/main/002_Parte/010/003_HF_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aplicaciones de Transformers en Procesamiento de Lenguaje Natural

Este cuaderno explora distintas aplicaciones de los modelos Transformers en tareas de procesamiento de lenguaje natural (PLN), utilizando ejemplos prácticos y explicaciones teóricas.

## 1. Definir el texto de ejemplo

A continuación, se presenta un texto de ejemplo que será utilizado en las distintas aplicaciones de modelos Transformers. Este texto simula una queja de un cliente sobre un pedido equivocado, lo que nos permitirá explorar tareas como clasificación, reconocimiento de entidades, respuesta a preguntas, resumen, traducción y generación de texto.

In [ ]:
texto = """Querido MercadoLibre, la semana pasada pedí una figura de acción de Optimus Prime desde su tienda online.
Para mi sorpresa, cuando abrí el paquete, descubrí horrorizado que me habían enviado una figura de Megatron.
Como fan de los Autobots, espero que entiendan mi decepción. Solicito un cambio urgente del producto."""

## 2. Clasificación de texto con Transformers

La **clasificación de texto** es una tarea fundamental en PLN que consiste en asignar una o varias etiquetas a un texto, como por ejemplo identificar el sentimiento (positivo, negativo, neutro) o la intención del mensaje. Los modelos Transformers, gracias a su capacidad para comprender el contexto, han demostrado un rendimiento sobresaliente en esta tarea.

Utilizaremos el pipeline `text-classification` de la librería 🤗 Transformers para analizar el texto de ejemplo y visualizar el resultado de la clasificación.

In [ ]:
from transformers import pipeline

In [ ]:
# Modelo de clasificación de texto (sentimiento) en español
# pysentimiento/robertuito-sentiment-analysis es robusto para el analisis de sentimiento en español.
classifier = pipeline("text-classification", model="pysentimiento/robertuito-sentiment-analysis")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/925 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/435M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
import pandas as pd

outputs = classifier(texto)
pd.DataFrame(outputs)

,label,score
0,NEG,0.913063


## 3. Reconocimiento de entidades nombradas (NER)

El **Reconocimiento de Entidades Nombradas** (NER, por sus siglas en inglés) es la tarea de identificar y clasificar automáticamente nombres propios, lugares, organizaciones y otras entidades relevantes dentro de un texto. Los Transformers permiten realizar NER de manera eficiente y precisa.

A continuación, aplicamos el pipeline `ner` al texto de ejemplo para extraer las entidades nombradas presentes.

In [ ]:
# Modelo de reconocimiento de entidades nombradas (NER) en español
# mrm8488/bert-spanish-cased-finetuned-ner está basado en BETO y finetunado para español.
ner_tagger = pipeline("ner", model="mrm8488/bert-spanish-cased-finetuned-ner", aggregation_strategy="simple")

In [ ]:
outputs = ner_tagger(texto)
pd.DataFrame(outputs)

In [ ]:
texto_2 = "Lionel Messi nació en Rosario, jugó en el FC Barcelona y ahora vive en Miami."

resultado = ner_tagger(texto_2)
pd.DataFrame(resultado)

## 4. Respuesta a preguntas basada en contexto

La tarea de **respuesta a preguntas** (Question Answering) consiste en responder preguntas específicas utilizando un contexto dado. Los modelos Transformers pueden comprender el texto y extraer la información relevante para responder de manera precisa.

En el siguiente ejemplo, preguntamos al modelo qué desea el cliente, utilizando el pipeline `question-answering`.

In [ ]:
# Modelo de respuesta a preguntas en español
# PlanTL-GOB-ES/roberta-large-bne-sqac es de PlanTL-GOB-ES, entrenado en SQAC.
reader = pipeline("question-answering", model="PlanTL-GOB-ES/roberta-large-bne-sqac")

In [ ]:
question = "¿Qué quiere el cliente?"
outputs = reader(question=question, context=texto)
pd.DataFrame([outputs])

In [ ]:
lector = pipeline("question-answering", model="mrm8488/bert-base-spanish-wwm-cased-finetuned-spa-squad2-es")

contexto = """Querido MercadoLibre, la semana pasada pedí una figura de acción de Optimus Prime desde su tienda online.
Para mi sorpresa, cuando abrí el paquete, descubrí horrorizado que me habían enviado una figura de Megatron.
Como fan de los Autobots, espero que entiendan mi decepción. Solicito un cambio urgente del producto."""
pregunta = "¿Qué quiere el cliente?"

respuesta = lector(question=pregunta, context=contexto)
pd.DataFrame([respuesta])

## 5. Resumen automático de texto

El **resumen automático** permite condensar la información principal de un texto extenso en una versión más corta, manteniendo el significado esencial. Los Transformers han revolucionado esta tarea gracias a su capacidad de comprensión contextual.

A continuación, utilizamos el pipeline `summarization` para obtener un resumen del texto de ejemplo.

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [ ]:
# Cargar modelo y tokenizer
model_name = "csebuetnlp/mT5_multilingual_XLSum"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [ ]:
# Crear el pipeline
resumidor = pipeline("summarization", model=model, tokenizer=tokenizer)

In [ ]:
# Texto para resumir
texto = """Querido MercadoLibre, la semana pasada pedí una figura de acción de Optimus Prime desde su tienda online.
Para mi sorpresa, cuando abrí el paquete, descubrí horrorizado que me habían enviado una figura de Megatron.
Como fan de los Autobots, espero que entiendan mi decepción. Solicito un cambio urgente del producto"""

In [ ]:
# Generar resumen
resumen = resumidor(texto, max_length=80, min_length=20, do_sample=False)
print(resumen[0]['summary_text'])

## 6. Traducción automática del español a inglés

La **traducción automática** es una de las aplicaciones más destacadas de los Transformers, permitiendo traducir textos entre diferentes idiomas con alta calidad. Utilizaremos el pipeline `translation_es_to_en` para traducir el texto de ejemplo del inglés al alemán.

In [ ]:
# Modelo de traducción de español a inglés
# Helsinki-NLP/opus-mt-es-en es un modelo estándar y confiable para esta tarea.
translator = pipeline("translation_es_to_en",
                      model="Helsinki-NLP/opus-mt-es-en")

In [ ]:
outputs = translator(texto, clean_up_tokenization_spaces=True, min_length=130)
print(outputs[0]['translation_text'])

In [ ]:
traductor = pipeline("translation", model="Helsinki-NLP/opus-mt-es-de")

In [ ]:
texto = "La inteligencia artificial está transformando la forma en que vivimos y trabajamos."
traduccion = traductor(texto)
print(traduccion[0]['translation_text'])

## 7. Generación automática de texto (respuesta de servicio al cliente)

La **generación automática de texto** permite crear o continuar textos de manera coherente a partir de un prompt inicial. Esta capacidad es útil, por ejemplo, para redactar respuestas automáticas en atención al cliente.

En el siguiente ejemplo, generamos una posible respuesta de servicio al cliente utilizando el pipeline `text-generation`.

In [ ]:
# Modelo de generación de texto en español
# DeepESP/gpt2-spanish es un GPT-2 entrenado desde cero con texto y un tokenizer en español.
generator = pipeline("text-generation", model="DeepESP/gpt2-spanish")

In [ ]:
response = "Estimado Bumblebee, lamento mucho saber que su pedido se mezcló."
prompt = texto + "\n\nRespuesta del servicio al cliente:\n" + response
outputs = generator(prompt, max_length=100, do_sample=True, temperature=1.0) # Añadir parámetros para mejor generación
print(outputs[0]['generated_text'])

In [ ]:
from transformers import pipeline

# Modelo en español
generador = pipeline("text-generation", model="datificate/gpt2-small-spanish")

# Texto del cliente
texto = """Querido MercadoLibre, la semana pasada pedí una figura de acción de Optimus Prime desde su tienda online.
Para mi sorpresa, cuando abrí el paquete, descubrí horrorizado que me habían enviado una figura de Megatron.
Como fan de los Autobots, espero que entiendan mi decepción. Solicito un cambio urgente del producto."""

# Primer renglón de la respuesta
respuesta_inicial = "Estimado Bumblebee, lamentamos mucho lo ocurrido con su pedido. "

# Prompt claro, sin repetir encabezado
prompt = texto + "\n\nRespuesta del servicio al cliente:\n" + respuesta_inicial

# Generación
outputs = generador(
    prompt,
    max_new_tokens=150,
    do_sample=True,
    temperature=1.0,
    top_k=50,
    top_p=0.9,
    repetition_penalty=1.3,
    eos_token_id=50256  # token de final en GPT-2
)

# Mostrar resultado
print(outputs[0]['generated_text'])